In [1]:
import pandas as pd
import geopandas as gpd
import folium

# read the shape file
#sf = gpd.read_file("../data/raw/POA_2021_AUST_GDA94.shp")
sf = gpd.read_file("../data/shape/POA_2021_AUST_GDA2020.shp")
sf = sf.iloc[672:1366]

# Convert the geometry shaape to to latitude and longitude
# Please attribute this if you are using it
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf.rename(columns={'POA_CODE21': 'Postcode'})

# create a JSON 
geoJSON = sf[['Postcode', 'geometry']].drop_duplicates('Postcode').to_json()


In [2]:
# Population Distribution
df = pd.read_csv('../data/curated/top_suburb5.csv')
sf['Postcode']=sf['Postcode'].astype(int).astype('str')
df['addressPostcode'] = df['addressPostcode'].astype(int).astype('str')
df = df.merge(sf[['Postcode', 'geometry']], left_on='addressPostcode', right_on='Postcode')

In [3]:
df.head()

,addressSuburb,addressPostcode,IEO_Score,min_distance_to_shopping_mall,min_distance_to_secondary_school,min_distance_to_park,min_walking_distance_to_station,medical_institution_count,price,IER_Score,...,min_distance_to_park_normalized,min_walking_distance_to_station_normalized,medical_institution_count_normalized,price_normalized,IER_Score_normalized,liveability_score,affordability_score,overall_score,Postcode,geometry
0,KEW,3101,1159.67,4.453438,0.864792,0.614896,1.079500,15.520833,805.468750,1058.24,...,0.908361,0.937117,0.715278,0.715036,1.000000,0.875444,0.857518,0.866481,3101,"POLYGON ((145.04052 -37.79016, 145.04065 -37.7..."
1,HAWTHORN EAST,3123,1164.25,1.150732,0.894756,1.011951,0.695354,14.000000,578.231707,1013.99,...,0.849188,0.959981,0.745098,0.816936,0.854134,0.894025,0.835535,0.864780,3123,"POLYGON ((145.05746 -37.82528, 145.05736 -37.8..."
2,YARRAVILLE,3013,1115.25,3.758636,0.928182,0.649091,0.859125,7.420455,650.227273,1035.13,...,0.903265,0.950234,0.874109,0.784651,0.923820,0.862866,0.854236,0.858551,3013,"POLYGON ((144.88986 -37.80977, 144.88992 -37.8..."
3,TOORAK,3142,1176.83,5.674918,0.661639,0.553689,0.833230,27.180328,700.983607,1034.95,...,0.917483,0.951775,0.486660,0.761891,0.923227,0.855476,0.842559,0.849017,3142,"POLYGON ((145.01263 -37.83051, 145.01265 -37.8..."
4,HAWTHORN,3122,1169.12,7.105345,0.993190,0.388276,1.295060,13.750000,570.043103,989.66,...,0.942135,0.924287,0.750000,0.820608,0.773932,0.878854,0.797270,0.838062,3122,"POLYGON ((145.04540 -37.82337, 145.04500 -37.8..."


In [ ]:
# plot the counts on geo map
m = folium.Map(location=[-36.9848, 143.3906], zoom_start=10)
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data= df, # data source
    columns=['Postcode','overall_score'], # the columns required
    key_on='properties.Postcode', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='white',
    legend_name='overall_score Distribution'
)
c.add_to(m)

# save the map
m.save('../plots/overall_score.html')

m